# Magic DB
## Data Engineering Capstone Project

### Project Summary

Magic: The Gathering (MTG) is a popular card game from the 90s that has a solid fan base and active trading card community. The game competitiveness and complexity attracts fans all around the world, creating a high demand for cards in the market.

The card trading market for MTG is complex given that number of players, the geographical spread of the game and the professional scenario. Even the construction of a new deck by the fan base can sky rocket the price of a single card within a day. In addition, official cards are released seasonally and many game stores buy and sell MTG products to casual or professional players making card prices volatile.

In the following sections we will explore the data and explain the steps taken.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

> **NOTE:**
This is a unoffical project for academic purpose only and should not be used for monetary gain. It is not funded or endorsed by any company.

In [62]:
# All imports
import os
import pandas as pd
import gzip
import json
import requests

from collections.abc import Mapping
from operator import add

### Step 1: Scope the Project and Gather Data

#### Scope
This project proposes the construction of a Magic database with card dimensional data and the prices that changes every day. This information is gathered and optimized for a fictional Data Science team to utilize in order to predict card prices in the next days. 

To accomplish this, we created an Airflow pipeline that extract the data from public hosts [Scryfall](https://scryfall.com) and [MTGJson](https://mtgjson.com), creates the source datasets for the project, load the dataset into staging tables in Redshift and finally populates the dimension and fact tables in a star schema also in Redshift. The schema is created to optimize card price queries.

#### Describe and Gather Data
In this project, we will collect data from two distinct sources:

- [Scryfall](https://scryfall.com): following the [guidelines](https://scryfall.com/docs/api) of [MTG policy](https://company.wizards.com/en/legal/fancontentpolicy) this site provides an API to search MTG cards and detailed information, even the card images are available to request. In this project, we will request programmatically the [bulk data](https://scryfall.com/docs/api/bulk-data) with all the card.

- [MTGJson](https://mtgjson.com): for the card prices, we resource to MTGJson that provides a download link to all card prices that they collect from major stores in Europe and United States. Their guidlines and licesing are available [here](https://github.com/mtgjson/mtgjson).

#### Scryfall
Lets first collect the data from Scryfall and have a look at it. To do this, we will request the API for the bulk-data endpoint:

In [2]:
response = requests.get("https://api.scryfall.com/bulk-data")
response.json()['data']

[{'object': 'bulk_data',
  'id': '27bf3214-1271-490b-bdfe-c0be6c23d02e',
  'type': 'oracle_cards',
  'updated_at': '2021-12-18T22:04:10.398+00:00',
  'uri': 'https://api.scryfall.com/bulk-data/27bf3214-1271-490b-bdfe-c0be6c23d02e',
  'name': 'Oracle Cards',
  'description': 'A JSON file containing one Scryfall card object for each Oracle ID on Scryfall. The chosen sets for the cards are an attempt to return the most up-to-date recognizable version of the card.',
  'compressed_size': 13178704,
  'download_uri': 'https://c2.scryfall.com/file/scryfall-bulk/oracle-cards/oracle-cards-20211218220410.json',
  'content_type': 'application/json',
  'content_encoding': 'gzip'},
 {'object': 'bulk_data',
  'id': '6bbcf976-6369-4401-88fc-3a9e4984c305',
  'type': 'unique_artwork',
  'updated_at': '2021-12-18T22:13:46.693+00:00',
  'uri': 'https://api.scryfall.com/bulk-data/6bbcf976-6369-4401-88fc-3a9e4984c305',
  'name': 'Unique Artwork',
  'description': 'A JSON file of Scryfall card objects that t

In [3]:
# Lets see all the data types:
[x['type'] for x in response.json()['data']]

['oracle_cards', 'unique_artwork', 'default_cards', 'all_cards', 'rulings']

In [4]:
# Now we get only the 'all_cards' because it is the most complete data.
[x for x in response.json()['data'] if x['type'] == 'all_cards']

[{'object': 'bulk_data',
  'id': '922288cb-4bef-45e1-bb30-0c2bd3d3534f',
  'type': 'all_cards',
  'updated_at': '2021-12-18T22:12:30.447+00:00',
  'uri': 'https://api.scryfall.com/bulk-data/922288cb-4bef-45e1-bb30-0c2bd3d3534f',
  'name': 'All Cards',
  'description': 'A JSON file containing every card object on Scryfall in every language.',
  'compressed_size': 212614165,
  'download_uri': 'https://c2.scryfall.com/file/scryfall-bulk/all-cards/all-cards-20211218221230.json',
  'content_type': 'application/json',
  'content_encoding': 'gzip'}]

In [5]:
# With the download_uri we can gather the JSON data
scryfall_json = requests.get('https://c2.scryfall.com/file/scryfall-bulk/all-cards/all-cards-20211216221311.json')
scryfall_json = scryfall_json.json()

In [6]:
# The data returned is a list with all the information per card.
print('Number of Columns: ', len(scryfall_json[0].keys()))
print('Data type', type(scryfall_json))
print('Total rows', len(scryfall_json))

Number of Columns:  62
Data type <class 'list'>
Total rows 349905


In [52]:
# Lets make the data smaller because Redshift only accepts 4Mb data
def select_columns(data):
    field_list = ['id', 'name', 'lang', 'released_at', 'layout', 'mana_cost', 'cmc',
                  'type_line', 'oracle_text', 'power', 'toughness', 'colors', 'color_identity',
                  'keywords', 'legalities', 'reserved', 'foil', 'nonfoil', 'oversized', 'promo',
                  'reprint', 'variation', 'set_id', 'set', 'set_name', 'set_type', 'collector_number',
                  'digital', 'rarity', 'artist', 'artist_ids', 'border_color', 'frame', 'full_art',
                  'textless', 'booster', 'story_spotlight', 'printed_name', 'printed_type_line',
                  'printed_text', 'security_stamp', 'loyalty', 'watermark', 'produced_mana',
                  'color_indicator', 'tcgplayer_etched_id', 'content_warning', 'life_modifier',
                  'hand_modifier']
    return({key: data.get(key) for key in field_list})


small_scryfall_json = [select_columns(data) for data in scryfall_json]

In [25]:
# The data returned after filtering the data.
print('Number of Columns: ', len(small_scryfall_json[0].keys()))
print('Data type', type(small_scryfall_json))
print('Total rows', len(small_scryfall_json))


Number of Columns:  49
Data type <class 'list'>
Total rows 349905


In [56]:
from tqdm import tqdm

filename = 'scryfall_json.json.gz'
if os.path.isfile(filename):
    os.remove(filename)

for line in tqdm(small_scryfall_json, total=len(small_scryfall_json)):
    with gzip.open(filename, "at", encoding='utf-8') as file:
        string = json.dumps(line, ensure_ascii=False)
        file.write(string + "\n")

100%|██████████| 349905/349905 [03:45<00:00, 1554.48it/s]


Now that we have MTG card informations, lets collect the prices

#### MTGJson
This dataset we have to make some transformations to meet our goals. First lets collect two separate data from the MTGJson endpoint:

- `Prices`: https://mtgjson.com/api/v5/AllPrices.json
- `Prints`: https://mtgjson.com/api/v5/AllPrintings.json

In [ ]:
# Download prices data
prices = requests.get("https://mtgjson.com/api/v5/AllPrices.json")
prices = prices.json()
prices = prices['data']

# Download prints data
prints = requests.get("https://mtgjson.com/api/v5/AllPrintings.json")
prints = prints.json()
prints = prints['data']

The `prices` data is a complex nested dictionary that we need to acces further to find the prices. First lets explore all the possible keys in the dictionary and a sample of its data.

In [ ]:
# first key is a card id
list(prices.keys())[0:10]

['00010d56-fe38-5e35-8aed-518019aa36a5',
 '0001e0d0-2dcd-5640-aadc-a84765cf5fc9',
 '0003caab-9ff5-5d1a-bc06-976dd0457f19',
 '0003d249-25d9-5223-af1e-1130f09622a7',
 '0004a4fb-92c6-59b2-bdbe-ceb584a9e401',
 '00054115-b2b6-5e22-a694-76fc8639eeb2',
 '00059c8d-868a-53ef-a1b0-fcfaabed2570',
 '0005d268-3fd0-5424-bc6b-573ecd713aa1',
 '0005f481-f2d4-53fa-ba37-cfcf5a5f87f1',
 '0006172e-304e-5f7b-ba48-f21b8da92178']

In [ ]:
# SECOND level keys, lets call it `online_paper`
aux = []
for key in prices:
    aux.extend(prices.get(key).keys())
set(aux)

{'mtgo', 'paper'}

In [ ]:
# FIRST level keys, lets call it `online_paper`
aux1, aux2, aux3, aux4, aux5 = [], [], [], [], []
for key1 in prices:
    aux1.extend(prices.get(key1))
    for key2 in prices.get(key1):
        aux2.extend(prices.get(key1).get(key2).keys())
        for key3 in prices.get(key1).get(key2):
            aux3.extend(prices.get(key1).get(key2).get(key3).keys())
            for key4 in prices.get(key1).get(key2).get(key3):
                if(key4 == 'currency'):
                    aux4.extend([prices.get(key1).get(
                        key2).get(key3).get(key4)])
                else:
                    aux4.extend(prices.get(key1).get(
                        key2).get(key3).get(key4).keys())

                    for key5 in prices.get(key1).get(key2).get(key3).get(key4):
                        aux5.extend(prices.get(key1).get(key2).get(
                            key3).get(key4).get(key5).keys())

print(100*'-')
print('First level')
print(set(aux1))

print(100*'-')
print('Second level')
print(set(aux2))

print(100*'-')
print('Third level')
print(set(aux3))

print(100*'-')
print('Fourth level')
print(set(aux4))

print(100*'-')
print('Fifth level')
print(sorted(set(prices.get(key1).get(key2).get(
    key3).get(key4).get(key5))))


----------------------------------------------------------------------------------------------------
First level
{'paper', 'mtgo'}
----------------------------------------------------------------------------------------------------
Second level
{'tcgplayer', 'cardkingdom', 'cardhoarder', 'cardmarket'}
----------------------------------------------------------------------------------------------------
Third level
{'retail', 'buylist', 'currency'}
----------------------------------------------------------------------------------------------------
Fourth level
{'normal', 'foil', 'EUR', 'USD'}
----------------------------------------------------------------------------------------------------
Fifth level
['2021-09-16', '2021-09-17', '2021-09-18', '2021-09-19', '2021-09-20', '2021-09-21', '2021-09-22', '2021-09-23', '2021-09-24', '2021-09-25', '2021-09-26', '2021-09-27', '2021-09-28', '2021-09-29', '2021-09-30', '2021-10-01', '2021-10-02', '2021-10-05', '2021-10-06', '2021-10-07', '2021-10-

In [ ]:
# Checking currency for each store
d = {}
d['tcgplayer'] = []
d['cardhoarder'] = []
d['cardkingdom'] = []
d['cardmarket'] = []

for key1 in prices:
    for key2 in prices.get(key1):
        for key3 in prices.get(key1).get(key2):
            for key4 in prices.get(key1).get(key2).get(key3):
                if(key4 == 'currency'):
                    d[key3].extend([prices.get(key1).get(
                        key2).get(key3).get(key4)])

d['tcgplayer']   = set(d['tcgplayer'])
d['cardhoarder'] = set(d['cardhoarder'])
d['cardkingdom'] = set(d['cardkingdom'])
d['cardmarket']  = set(d['cardmarket'])
d

{'tcgplayer': {'USD'},
 'cardhoarder': {'USD'},
 'cardkingdom': {'USD'},
 'cardmarket': {'EUR'}}

Lets unnest the JSON in the following format:

```
  card_id: card id for MTGJSON.
    values: uuid

  online_paper: indicates if it is the price of paper or online card
    values: {'mtgo', 'paper'}
    
  store: store that the price was extracted.
    values: {'tcgplayer', 'cardhoarder', 'cardkingdom', 'cardmarket'}

  price_type: indicates if price is a buylist (similar to buy bid price) or retail.
    values: {'retail', 'buylist'}

  currency: price currency. Here only 'cardmarket' is in EUR.
    values: {'USD', 'EUR'}

  card_type: indicates if the card is normal or foil.
    values: {'foil', 'normal'}
```

In [ ]:
# Auxiliar function to unnest a dictionary
def flattenDict(d, join=add, lift=lambda x: (x,)):
    results = []
    _FLAG_FIRST = object()

    def visit(subdict, results, partialKey):
        for k, v in subdict.items():
            newKey = lift(k) if partialKey == _FLAG_FIRST else join(
                partialKey, lift(k))
            if isinstance(v, Mapping):
                visit(v, results, newKey)
            else:
                results.append(add(newKey, lift(v)))
    visit(d, results, _FLAG_FIRST)
    return results

currency = {
    'tcgplayer':  'USD',
    'cardmarket':  'EUR',
    'cardkingdom':  'USD',
    'cardhoarder':  'USD'
}
columns = ["card_id", "online_paper", "store", "price_type",
            "card_type", "dt", "price"]

df = pd.DataFrame(flattenDict(prices), columns=columns)
df = df[~df.card_type.isin(['USD', 'EUR'])]
df["currency"] = df.store.map(currency)
df.head()

,card_id,online_paper,store,price_type,card_type,dt,price,currency
0,00010d56-fe38-5e35-8aed-518019aa36a5,paper,cardkingdom,buylist,foil,2021-09-16,4.8,USD
1,00010d56-fe38-5e35-8aed-518019aa36a5,paper,cardkingdom,buylist,foil,2021-09-17,4.8,USD
2,00010d56-fe38-5e35-8aed-518019aa36a5,paper,cardkingdom,buylist,foil,2021-09-18,4.8,USD
3,00010d56-fe38-5e35-8aed-518019aa36a5,paper,cardkingdom,buylist,foil,2021-09-19,4.8,USD
4,00010d56-fe38-5e35-8aed-518019aa36a5,paper,cardkingdom,buylist,foil,2021-09-20,4.8,USD


In [ ]:
# saving the dataset
df.to_csv('mtgjson_prices.csv.gz', compression='gzip', index=False)

Now lets get the card names from `prints` to be able to merge data with the Scryfall dataset.

In [ ]:
# first key is the card edition. The edition is a set in which a group of cards
# is released together and share some caracteristics
list(prints.keys())[0:10]

['10E', '2ED', '2XM', '3ED', '4BB', '4ED', '5DN', '5ED', '6ED', '7ED']

In [ ]:
# All the data from the edition
prints.get('10E').keys()

dict_keys(['baseSetSize', 'block', 'booster', 'cards', 'code', 'isFoilOnly', 'isOnlineOnly', 'keyruneCode', 'mcmId', 'mcmName', 'mtgoCode', 'name', 'releaseDate', 'sealedProduct', 'tcgplayerGroupId', 'tokens', 'totalSetSize', 'translations', 'type'])

In [ ]:
# To simplify the project, lets only get the card info
# each edition has a set of cards. Each card have the following information
prints.get('10E').get('cards')[0].keys()

dict_keys(['artist', 'availability', 'borderColor', 'colorIdentity', 'colors', 'convertedManaCost', 'edhrecRank', 'finishes', 'foreignData', 'frameVersion', 'hasFoil', 'hasNonFoil', 'identifiers', 'isReprint', 'keywords', 'layout', 'legalities', 'manaCost', 'manaValue', 'name', 'number', 'originalText', 'originalType', 'power', 'printings', 'purchaseUrls', 'rarity', 'rulings', 'setCode', 'subtypes', 'supertypes', 'text', 'toughness', 'type', 'types', 'uuid', 'variations'])

In [46]:
# Lets loop through the `edition` and get the `id`, `name`, `collector_number`
# these information will be important to join scryfall and mtgjson data
columns = ['card_id', 'name', 'collector_number', 'edition']

df = pd.DataFrame(columns=columns)
for edition in prints:
    cards = prints.get(edition).get('cards')
    if(len(cards) > 0):
        aux = pd.DataFrame(cards)
        aux = aux[['uuid', 'name', 'number']]
        aux['edition'] = edition
        aux.columns = columns
        df = pd.concat([df, aux], axis=0)

In [47]:
# dataset with card_name, name, edition
df.head()

,card_id,name,collector_number,edition
0,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,Ancestor's Chosen,1,10E
1,57aaebc1-850c-503d-9f6e-bb8d00d8bf7c,Angel of Mercy,2,10E
2,8ac972b5-9f6e-5cc8-91c3-b9a40a98232e,Aven Cloudchaser,7,10E
3,a69b404f-144a-5317-b10e-7d9dce135b24,Ballista Squad,8,10E
4,6d268c95-c176-5766-9a46-c14f739aba1c,Bandage,9,10E


In [ ]:
# saving the dataset
df.to_csv('mtgjson_prints.csv.gz', compression='gzip', index=False)

### Step 2: Explore and Assess the Data

#### Explore the Data 
The data comes from thre well governed datasets:
- `scryfall_json.json.gz`: has ~ 350_000 rows and card information.
- `mtgjson_prices.csv.gz`: has ~ 28_000_000 rows and all card prices
- `mtgjson_prints.csv.gz`: has ~ 64_000 rows and all contains the cards names and edition of the mtgjson_prices dataset.

In [19]:
def get_df_information(df):
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    df_final = pd.DataFrame()
    for column in df.columns:
        aux = pd.DataFrame()
        aux['columns'] = [column]
        aux['dtpye']   = df[column].dtype
        aux['%duplicates'] = df[column].duplicated().sum()/df.shape[0]
        aux['%null']   = df[column].isnull().sum()/df.shape[0]
        aux['sample'] = ' | '.join(
            df[column][~df[column].isnull()].astype(str).unique()[0:5])
        df_final = pd.concat([df_final, aux], axis=0)
    return(df_final.reset_index(drop=True))


##### Lets start with `scryfall_json.json.gz`

In [58]:
# Lets create a DataFrame and have a look.
scryfall_df = pd.read_json('scryfall_json.json.gz',
                           compression='gzip', lines=True)


In [59]:
print('Shape: ', scryfall_df.shape)
scryfall_df.head()

Shape:  (349905, 49)


,id,name,lang,released_at,layout,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,legalities,reserved,foil,nonfoil,oversized,promo,reprint,variation,set_id,set,set_name,set_type,collector_number,digital,rarity,artist,artist_ids,border_color,frame,full_art,textless,booster,story_spotlight,printed_name,printed_type_line,printed_text,security_stamp,loyalty,watermark,produced_mana,color_indicator,tcgplayer_etched_id,content_warning,life_modifier,hand_modifier
0,0000579f-7b35-4ed3-b44c-db2a538066fe,Fury Sliver,en,2006-10-06,normal,{5}{R},6.0,Creature — Sliver,All Sliver creatures have double strike.,3,3,[R],[R],[],"{'standard': 'not_legal', 'future': 'not_legal...",False,True,True,False,False,False,False,c1d109bc-ffd8-428f-8d7d-3f8d7e648046,tsp,Time Spiral,expansion,157,False,uncommon,Paolo Parente,[d48dd097-720d-476a-8722-6a02854ae28b],black,2003,False,False,True,False,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN
1,00006596-1166-4a79-8443-ca9f82e6db4e,Kor Outfitter,en,2009-10-02,normal,{W}{W},2.0,Creature — Kor Soldier,"When Kor Outfitter enters the battlefield, you...",2,2,[W],[W],[],"{'standard': 'not_legal', 'future': 'not_legal...",False,True,True,False,False,False,False,eb16a2bd-a218-4e4e-8339-4aa1afc0c8d2,zen,Zendikar,expansion,21,False,common,Kieran Yanner,[aa7e89ed-d294-4633-9057-ce04dacfcfa4],black,2003,False,False,True,False,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2,00009878-d086-46f0-a964-15734d8368ac,Spirit of the Hearth,fr,2017-08-25,normal,{4}{W}{W},6.0,Creature — Cat Spirit,Flying\nYou have hexproof. (You can't be the t...,4,5,[W],[W],[Flying],"{'standard': 'not_legal', 'future': 'not_legal...",False,False,True,False,False,True,False,5caec427-0c78-4c37-b4ec-30f7e0ba9abf,c17,Commander 2017,commander,73,False,rare,Jason Chan,[8062d5a9-51b6-4822-933f-fa9e9dba8416],black,2015,False,False,False,False,Esprit du foyer,Créature : chat et esprit,Vol\nVous avez la défense talismanique. (Vous ...,oval,None,None,None,None,NaN,NaN,NaN,NaN
3,0000a54c-a511-4925-92dc-01b937f9afad,Spirit,en,2015-05-22,token,,0.0,Token Creature — Spirit,Flying,1,1,[W],[W],[Flying],"{'standard': 'not_legal', 'future': 'not_legal...",False,False,True,False,False,True,False,f7aa47c6-c1e2-4de5-9a68-4406d84bd6bb,tmm2,Modern Masters 2015 Tokens,token,5,False,common,Mike Sass,[155bc2cb-038d-4b1f-9990-6178db1d1a21],black,2015,False,False,True,False,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN
4,0000cd57-91fe-411f-b798-646e965eec37,Siren Lookout,en,2017-09-29,normal,{2}{U},3.0,Creature — Siren Pirate,Flying\nWhen Siren Lookout enters the battlefi...,1,2,[U],[U],"[Flying, Explore]","{'standard': 'not_legal', 'future': 'not_legal...",False,True,True,False,False,False,False,fe0dad85-54bc-4151-9200-d68da84dd0f2,xln,Ixalan,expansion,78,False,common,Chris Rallis,[a8e7b854-b15a-421a-b66d-6e68187ae285],black,2015,False,False,True,False,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN


In [60]:
get_df_information(scryfall_df)

,columns,dtpye,%duplicates,%null,sample
0,id,object,0.000000,0.000000,0000579f-7b35-4ed3-b44c-db2a538066fe | 0000659...
1,name,object,0.930181,0.000000,Fury Sliver | Kor Outfitter | Spirit of the He...
2,lang,object,0.999951,0.000000,en | fr | pt | ja | ru
3,released_at,datetime64[ns],0.997934,0.000000,2006-10-06 | 2009-10-02 | 2017-08-25 | 2015-05...
4,layout,object,0.999943,0.000000,normal | token | split | transform | art_series
5,mana_cost,object,0.997605,0.008405,{5}{R} | {W}{W} | {4}{W}{W} | | {2}{U}
6,cmc,float64,0.999943,0.000014,6.0 | 2.0 | 0.0 | 3.0 | 1.0
7,type_line,object,0.992155,0.000014,Creature — Sliver | Creature — Kor Soldier | C...
8,oracle_text,object,0.937657,0.012441,All Sliver creatures have double strike. | Whe...
9,power,object,0.999900,0.521176,3 | 2 | 4 | 1 | 7


In [61]:
print("Scryfall is unique in name, set, collector number and language: ",
scryfall_df[['name', 'set', 'collector_number','lang']].duplicated().sum())

Scryfall is unique in name, set, collector number and language:  0


In [70]:
print("Scryfall is unique in card id: ",
      scryfall_df.id.duplicated().sum())

Scryfall is unique in card id:  0


##### Now lets have a look at  `mtgjson_prices.csv.gz` and `mtgjson_prints.csv.gz`

##### Prices

In [ ]:
# Lets create a DataFrame and have a look at mtgjson_prices.
mtgjson_prices = pd.read_csv('mtgjson_prices.csv.gz', compression='gzip')

In [ ]:
print('Shape: ', mtgjson_prices.shape)
mtgjson_prices.head()

Shape:  (28211466, 8)


,card_id,online_paper,store,price_type,card_type,dt,price,currency
0,00010d56-fe38-5e35-8aed-518019aa36a5,paper,cardkingdom,buylist,foil,2021-09-16,4.8,USD
1,00010d56-fe38-5e35-8aed-518019aa36a5,paper,cardkingdom,buylist,foil,2021-09-17,4.8,USD
2,00010d56-fe38-5e35-8aed-518019aa36a5,paper,cardkingdom,buylist,foil,2021-09-18,4.8,USD
3,00010d56-fe38-5e35-8aed-518019aa36a5,paper,cardkingdom,buylist,foil,2021-09-19,4.8,USD
4,00010d56-fe38-5e35-8aed-518019aa36a5,paper,cardkingdom,buylist,foil,2021-09-20,4.8,USD


In [ ]:
get_df_information(mtgjson_prices)

,columns,dtpye,%null,sample
0,card_id,object,0.0,00010d56-fe38-5e35-8aed-518019aa36a5 | 0001e0d...
1,online_paper,object,0.0,paper | mtgo
2,store,object,0.0,cardkingdom | cardmarket | tcgplayer | cardhoa...
3,price_type,object,0.0,buylist | retail
4,card_type,object,0.0,foil | normal
5,dt,object,0.0,2021-09-16 | 2021-09-17 | 2021-09-18 | 2021-09...
6,price,float64,0.0,4.8 | 5.2 | 5.5 | 6.0 | 7.99
7,currency,object,0.0,USD | EUR


##### Prints

In [41]:
# Lets create a DataFrame and have a look at mtgjson_prints.
mtgjson_prints = pd.read_csv('mtgjson_prints.csv.gz', compression='gzip')

In [42]:
print('Shape: ', mtgjson_prints.shape)
mtgjson_prints.head()

Shape:  (64455, 4)


,card_id,name,collector_number,edition
0,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,Ancestor's Chosen,1,10E
1,57aaebc1-850c-503d-9f6e-bb8d00d8bf7c,Angel of Mercy,2,10E
2,8ac972b5-9f6e-5cc8-91c3-b9a40a98232e,Aven Cloudchaser,7,10E
3,a69b404f-144a-5317-b10e-7d9dce135b24,Ballista Squad,8,10E
4,6d268c95-c176-5766-9a46-c14f739aba1c,Bandage,9,10E


In [43]:
get_df_information(mtgjson_prints)

,columns,dtpye,%null,sample
0,card_id,object,0.0,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c | 57aaebc...
1,name,object,0.0,Ancestor's Chosen | Angel of Mercy | Aven Clou...
2,collector_number,object,0.0,1 | 2 | 7 | 8 | 9
3,edition,object,0.0,10E | 2ED | 2XM | 3ED | 4BB


In [45]:
print("MTGJson Prints is unique in card id: ",
      mtgjson_prints['card_id'].duplicated().sum())

MTGJson Prints is unique in card id:  0


#### Cleaning Steps

The Scryfall data has some complex data format in some fields, for example, `legalities` is a dictionary. In addition, some columns of the dataset that were not usefull in our project were already removed to allow a better copy to Redshift. For exemple, `mtgo_id` is an id in the online game that should not be used in any price forecast model.

Some ids we should keep to find the correct price for each card. In this project we will create dimensions out of the dataset to optimize the proposed goal. We will first load all the data into Redshift and treat the data in there.

In the MTGJson data, we need to JOIN the two tables and them find the correspoding cards in Scryfall. Since all the data is gathered from APIs and websites that makes a pre cleaning and we are constructing the dataset, we believe it is not necessary to do any further cleaning in this step. We will treat some particularities of the dataset in Redshift after loading these as steage tables.

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

We propose a Snowflake schema with the following design:

![erd](images/erd.png)

We have 4 dimension tables and one fact. Here are the description of each table (please refer to 4.3 for each field description):

### Fact Table
`prices`: fact table with card prices by date. The prices can be for online or paper MTG.
 - prices_id, card_id, online_paper, store, price_type, card_type, currency, dt, price

### Dimension Tables
`sets`: information about the set of the card. Each card belongs to only one set.
- set_id, set, set_name, set_type

`artists`: artists that create the art of the card. The card can have more than one artist.
- artist_id, artist

`time`: auxiliary time table.
- dt, day, month, year

`cards`: information of all cards in MTG. Each card is a singlw row, even if there is the same card in a different language.
- card_id, card_name, card_lang, released_at, layout, mana_cost, cmc, type_line, oracle_text, power, toughness, colors, color_identity, keywords, standard_legal, pioneer_legal, modern_legal, legacy_legal, historic_legal, reserved, foil, nonfoil, oversized, promo, reprint, variation, set_id, collector_number, digital, rarity, artist_ids, border_color, frame, full_art, textless, booster, story_spotlight, printed_name, printed_type_line, printed_text, security_stamp, loyalty, watermark, produced_mana, color_indicator, content_warning, life_modifier, hand_modifier

#### 3.2 Mapping Out Data Pipelines

![pipeline](images/pipeline.png)

The pipeline can be separeted in 3 main steps. First we download the data from the sources, transform it and load into the Redshift. Then, we load the dimension and fact tables. Finally, we run data checks to ensure quality of the final tables. Lets have a look at each step.

##### Step 1.1

![pipe_1_1](images/pipe_1_1.png)

We get the MTGJSON price data from the ALLPrices link. Transform it into a CSV table with all prices and send to Redshift stage table `staging_prices`.

##### Step 1.2

![pipe_1_2](images/pipe_1_2.png)

The same is done to prints with the ALLPrints link. This table holds the card name and set of prices and will allow us to JOIN the dimension tables from Scryfall. The Redshift stage table is `staging_prints`.

##### Step 1.3

![pipe_1_3](images/pipe_1_3.png)

For the Scryfall data, first we request the API for the bulk-data link and then we download the JSON and COPY it to Redshift in the `staging_scryfall` stage table.

##### Step 2

![pipe_2](images/pipe_2.png)

After we load the staging tables, we perform ETL on it to creat the dimension and fact tables.

##### Step 3

![pipe_3](images/pipe_3.png)

Finally, the a quality check is run in the final tables to ensure quality of the data loaded. 

### Step 4: Run Pipelines to Model the Data

The data model is all done in Airflow. please refer to the DAG `mtg_dag.py` for all the pipeline code.

#### 4.1 Create the data model
It is important to create the tables before running the pipeline. We create each table as follows:

```sql
CREATE TABLE IF NOT EXISTS public.staging_scryfall (
    id                  VARCHAR(36)
  , name                VARCHAR
  , lang                VARCHAR(10)
  , released_at         TIMESTAMP
  , layout              VARCHAR(50)
  , mana_cost           VARCHAR
  , cmc                 INT
  , type_line           VARCHAR
  , oracle_text         VARCHAR
  , power               VARCHAR(10)
  , toughness           VARCHAR(10)
  , colors              VARCHAR
  , color_identity      VARCHAR
  , keywords            VARCHAR
  , standard_legal      VARCHAR(20)
  , pioneer_legal       VARCHAR(20)
  , modern_legal        VARCHAR(20)
  , legacy_legal        VARCHAR(20)
  , historic_legal      VARCHAR(20)
  , reserved            BOOLEAN
  , foil                BOOLEAN
  , nonfoil             BOOLEAN
  , oversized           BOOLEAN
  , promo               BOOLEAN
  , reprint             BOOLEAN
  , variation           BOOLEAN
  , set_id              VARCHAR(36)
  , set                 VARCHAR(20)
  , set_name            VARCHAR
  , set_type            VARCHAR
  , collector_number    VARCHAR(20)
  , digital             BOOLEAN
  , rarity              VARCHAR(20)
  , artist              VARCHAR
  , artist_ids          VARCHAR
  , border_color        VARCHAR(20)
  , frame               VARCHAR
  , full_art            BOOLEAN
  , textless            BOOLEAN
  , booster             BOOLEAN
  , story_spotlight     BOOLEAN
  , printed_name        VARCHAR
  , printed_type_line   VARCHAR
  , printed_text        VARCHAR
  , security_stamp      VARCHAR(20)
  , loyalty             VARCHAR(10)
  , watermark           VARCHAR(20)
  , produced_mana       VARCHAR
  , color_indicator     VARCHAR
  , tcgplayer_etched_id INT
  , content_warning     INT
  , life_modifier       INT
  , hand_modifier       INT
);

CREATE TABLE IF NOT EXISTS public.staging_prices (
    card_id      VARCHAR(36)
  , online_paper VARCHAR(20)
  , store        VARCHAR(20)
  , price_type   VARCHAR(20)
  , card_type    VARCHAR(20)
  , dt           TIMESTAMP
  , price        FLOAT
  , currency     VARCHAR(10)
);

CREATE TABLE IF NOT EXISTS public.staging_prints (
    card_id          VARCHAR(36)
  , name             VARCHAR
  , collector_number VARCHAR(20)
  , edition          VARCHAR(20)
);

CREATE TABLE IF NOT EXISTS public."sets" (
    set_id   VARCHAR(36)
  , set      VARCHAR(20)
  , set_name VARCHAR
  , set_type VARCHAR
  , PRIMARY KEY (set_id)
);

CREATE TABLE IF NOT EXISTS public.artists (
    artist_id VARCHAR
  , artist    VARCHAR
  , PRIMARY KEY (artist_id)
);

CREATE TABLE IF NOT EXISTS public.time (
    dt    VARCHAR
  , day   INT
  , month INT
  , year  INT
  , PRIMARY KEY (dt)
);

CREATE TABLE IF NOT EXISTS public.cards (
    card_id           VARCHAR(36)
  , card_name         VARCHAR
  , card_lang         VARCHAR(10)
  , released_at       TIMESTAMP
  , layout            VARCHAR(50)
  , mana_cost         VARCHAR
  , cmc               INT
  , type_line         VARCHAR
  , oracle_text       VARCHAR
  , power             VARCHAR(10)
  , toughness         VARCHAR(10)
  , colors            VARCHAR
  , color_identity    VARCHAR
  , keywords          VARCHAR
  , standard_legal    VARCHAR(20)
  , pioneer_legal     VARCHAR(20)
  , modern_legal      VARCHAR(20)
  , legacy_legal      VARCHAR(20)
  , historic_legal    VARCHAR(20)
  , reserved          BOOLEAN
  , foil              BOOLEAN
  , nonfoil           BOOLEAN
  , oversized         BOOLEAN
  , promo             BOOLEAN
  , reprint           BOOLEAN
  , variation         BOOLEAN
  , set_id            VARCHAR(36)
  , collector_number  VARCHAR(20)
  , digital           BOOLEAN
  , rarity            VARCHAR(20)
  , artist_ids        VARCHAR
  , border_color      VARCHAR(20)
  , frame             VARCHAR
  , full_art          BOOLEAN
  , textless          BOOLEAN
  , booster           BOOLEAN
  , story_spotlight   BOOLEAN
  , printed_name      VARCHAR
  , printed_type_line VARCHAR
  , printed_text      VARCHAR
  , security_stamp    VARCHAR(20)
  , loyalty           VARCHAR(10)
  , watermark         VARCHAR(20)
  , produced_mana     VARCHAR
  , color_indicator   VARCHAR
  , content_warning   INT
  , life_modifier     INT
  , hand_modifier     INT
  , PRIMARY KEY (card_id)
  , FOREIGN KEY (set_id) REFERENCES "sets"(set_id)
  , FOREIGN KEY (artist_ids) REFERENCES artists(artist_id)
);

CREATE TABLE IF NOT EXISTS public.prices (
    prices_id     varchar(36)
  , card_id       VARCHAR(36)
  , online_paper  VARCHAR(20)
  , store         VARCHAR(20)
  , price_type    VARCHAR(20)
  , card_type     VARCHAR(20)
  , currency      VARCHAR(10)
  , dt            TIMESTAMP
  , price         FLOAT
  , PRIMARY KEY (prices_id)
  , FOREIGN KEY (card_id) REFERENCES cards(card_id)
  , FOREIGN KEY (dt) REFERENCES "time"(dt)
);
```

#### 4.2 Data Quality Checks

![pipe_3_1](images/pipe_3_1.png)

The qualities checks consists of:

- `check_rows`: counts the number of rows and it has to be greater than 0.
- `check_pk`: checks if there is any primary key as NULL.
- `check_duplicated`: checks if there any duplicated row in the dimensions.

#### 4.3 Data dictionary 

Please refer to `Data Dictionary.md` to the description of each field in the database.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.